In [ ]:
# Google Colab: Instalar estos paquetes en cada sesion
# using Pkg
# Pkg.add(["ArchGDAL", "Rasters", "GeoDataFrames"])

In [ ]:
using Rasters, ArchGDAL, GeoDataFrames, Statistics, Plots

In [ ]:
# Leer raster COG desde URL
cog_url = "https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio10_1981-2010_V.2.1.tif"
r = Raster(cog_url, lazy=true)

In [ ]:
ecu = view(r, X(-81.0852191551 .. -75.1860925808), Y(-5.0186422241 .. 1.4629920115))

In [ ]:
# Resamplear raster a EPSG:32717 (UTM WGS84 17S)
ecu_utm = Rasters.resample(ecu, res=(1000,1000), method = :bilinear, crs=EPSG(32717))

In [ ]:
Plots.plot(
    ecu_utm,
    color=:thermal,
    title="Temperatura media (1981-2010)",
    xlabel="UTM X (m)", ylabel="UTM Y (m)", colorbar_title="°C",
    aspect_ratio = :equal,
    size=(800,600)
)

In [ ]:
# Importar flatgeobuf desde github
url = "https://github.com/geoskydev/intro-julia-2025/raw/refs/heads/main/datos/cantones-ecuador.fgb"
cantones_poligonos = GeoDataFrames.read(url)
first(cantones_poligonos, 5)

In [ ]:
# Estadísticas zonales: temperatura media anual por cantón
median_temp = DataFrame(median_temp=zonal(median, ecu_utm, of=cantones_poligonos; progress=false))
insertcols!(cantones_poligonos, :temp_mediana => median_temp.median_temp)
first(cantones_poligonos, 5)

In [ ]:
Plots.plot(
     cantones_poligonos.geometry, 
     fill_z = permutedims(cantones_poligonos.temp_mediana),
     color = :thermal,
     linewidth = 0.2,
     linecolor = :black,
     title = "Temperatura media (1981-2010) por canton",
     xlabel="UTM X (m)", ylabel="UTM Y (m)", colorbar_title = "°C",
     aspect_ratio = :equal,
     size=(800,600)
)

## STAC items

In [ ]:
using STAC, Rasters, ArchGDAL, Plots

In [ ]:
url = "https://datacube.services.geo.ca/stac/api"
cat = STAC.Catalog(url)

In [ ]:
keys(cat)

In [ ]:
subcat = cat["et-year"]

In [ ]:
cog_urls = [href(item["cog"]) for item in eachitem(subcat)]

In [ ]:
rasters_list = [Raster(r) for r in cog_urls]

In [ ]:
# Create a plot
p = plot(layout=(6,4), size=(1500, 1500))

for i in 1:length(rasters_list)
    plot!(p, rasters_list[i], subplot=i, title=i+1999, colorbar=false, xticks=false, yticks=false, xlabel="", ylabel="")
end

p

In [ ]:
plot(
    rasters_list[1],
    color=:thermal,
    title="Evapotranspiración 2014",
    xlabel="Longitud",
    ylabel="Latitud",
    xformatter=:none,
    yformatter=:none,
    margin=5Plots.mm,
    colorbar_title="mm/año",
    aspect_ratio=:equal,
    size=(800,600),
    dpi=200
)